In [2]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import crepe
from music21 import stream, note, meter, tempo, duration
from IPython.display import display
from ipywidgets import FileUpload

### Upload do Arquivo do Áudio

In [5]:
uploader = FileUpload(accept='.wav', multiple=False)  # Only accept .wav files
display(uploader)

FileUpload(value={}, accept='.wav', description='Upload')

In [6]:
def save_uploaded_file(uploader):
    if uploader.value:
        file_name = list(uploader.value.keys())[0]
        with open(file_name, 'wb') as f:
            f.write(uploader.value[file_name]['content'])
        print(f"File {file_name} uploaded and saved!")
        return file_name
    else:
        print("No file uploaded!")
        return None

arquivo_audio = save_uploaded_file(uploader)

File photograph_teste02.wav uploaded and saved!


### Pré Processamento

In [ ]:
y, sr = sf.read(arquivo_audio, always_2d=False)
y = y[:, 0] if y.ndim > 1 else y

#  Retirar ruídos do áudio
non_silent_intervals = librosa.effects.split(y, top_db=30)  # Adjust `top_db` for sensitivity
y_denoised = np.concatenate([y[start:end] for start, end in non_silent_intervals])

#print(f"Shape after denoising: {y_denoised.shape}")

# Normalização do áudio
y_normalized = librosa.util.normalize(y_denoised)

#print(f"Max amplitude after normalization: {np.max(y_normalized)}")

# Resample Audio

y_resampled = librosa.resample(y_normalized, orig_sr=sr, target_sr=16000)

#print(f"Shape after resampling: {y_resampled.shape}, Resampled Rate: 16000 Hz")

### Detecção das Notas

In [ ]:
def detect_pitches_with_crepe(y_resampled, sr=16000):
    # Run CREPE pitch detection (it returns time, frequency, and amplitude of each pitch)
    output = crepe.predict(y_resampled, sr)

    time = output[0]
    frequency = output[1]

    # You can print out the first few detected pitches for inspection
    print("Detected pitch frequencies (in Hz):")
    print(frequency[:10])  # Print the first 10 detected frequencies

    return time, frequency

In [ ]:
def freq_to_note_with_music21(frequency):
    if frequency <= 0:  # Handle invalid frequencies or rests
        return "rest"
    
    p = pitch.Pitch()
    p.frequency = frequency
    return p.nameWithOctave  # Returns note name (e.g., "A4")

def map_crepe_output_to_notes_with_durations(times, frequencies):
    notes_with_durations = []
    current_note = None
    current_start_time = None

    for i in range(len(frequencies)):
        frequency = frequencies[i]
        time = times[i]

        # Convert frequency to note
        note_name = freq_to_note_with_music21(frequency)

        if note_name == current_note:
            # Continue sustaining the current note
            continue
        else:
            # If the note changes, finalize the previous note
            if current_note is not None:
                duration = time - current_start_time
                notes_with_durations.append((current_note, duration))

            # Start a new note
            current_note = note_name
            current_start_time = time

    # Add the final note
    if current_note is not None:
        duration = times[-1] - current_start_time
        notes_with_durations.append((current_note, duration))

    return notes_with_durations


In [ ]:
# Detectar Pitch
time, frequency = detect_pitches_with_crepe(y_resampled, sr=16000)

plt.figure(figsize=(10, 6))
plt.plot(time, frequency)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('Detected Pitch Frequencies Over Time')
plt.show()

# Mapear frequencias em notas musicais
notes_with_durations = map_crepe_output_to_notes_with_durations(times, frequencies)
print("Detected notes:")
print(notes)

### Gerar Partitura

In [ ]:
def generate_sheet_music(notes_with_durations, output_filename="sheet_music"):
    
    score = stream.Score()
    part = stream.Part()
    part.append(meter.TimeSignature("4/4"))
    part.append(tempo.MetronomeMark(number=120))

    for note_name, duration_in_seconds in notes_with_durations:
        # Convert duration to a quarter-length value
        # Assuming 120 BPM (2 seconds = 1 whole note, 1 second = half note, etc.)
        quarter_length = duration_in_seconds * 2

        if note_name == "rest":
            n = note.Rest(quarterLength=quarter_length)
        else:
            n = note.Note(note_name)
            n.quarterLength = quarter_length

        part.append(n)

    score.append(part)
    output_file_path = f"{output_filename}.pdf"
    score.write("pdf", output_file_path)
    print(f"Sheet music saved as {output_file_path}")

In [ ]:
generate_sheet_music(notes, output_filename="my_piano_sheet")